# Robot Demo (Working Motors)
Demos movement (wheel-level), eyes, camera, and TTS using your **Python 3.10** libraries.

Notes:
- Movement uses your working wheel-level motor stream (`/ros_robot_controller/set_motor_speeds`).
- We **do not** run cmd_vel drift here (that requires a live `/cmd_vel` subscriber and is a separate demo).


In [1]:
import os, sys, importlib, time, subprocess

mods = ("robot_moves", "eyes_lib", "camera_lib", "tts_lib")
print("Python:", sys.version.split()[0])
for m in mods:
    try:
        __import__(m)
        print("✔ imported", m)
    except Exception as e:
        print("✖ import failed for", m, "->", e)
        raise


In [2]:
# Quick ROS sanity checks (optional but handy)
def sh(cmd):
    try:
        out = subprocess.check_output(cmd, stderr=subprocess.STDOUT, text=True)
        return out.strip()
    except Exception as e:
        return f"(could not run {' '.join(cmd)}): {e}"

print(sh(["bash","-lc","source /opt/ros/humble/setup.bash >/dev/null 2>&1 || true; ros2 topic list | egrep -i 'cmd_vel|set_motor_speeds' || true"]))
print(sh(["bash","-lc","source /opt/ros/humble/setup.bash >/dev/null 2>&1 || true; ros2 topic info -v /ros_robot_controller/set_motor_speeds || true"]))


In [3]:
import robot_moves as rm
importlib.reload(rm)

# Tune these if needed
rm.set_base_speed(260)   # try 220..320 depending on surface
rm.set_rate(20)          # 15..30 is fine

print("Movement demo (wheel-level motor stream)")
print("Forward")
rm.forward(0.6); rm.stop(); time.sleep(0.25)

print("Backward")
rm.backward(0.6); rm.stop(); time.sleep(0.25)

print("Turn left")
rm.turn_left(0.5); rm.stop(); time.sleep(0.25)

print("Turn right")
rm.turn_right(0.5); rm.stop(); time.sleep(0.25)

print("Strafe left")
rm.left(0.6); rm.stop(); time.sleep(0.25)

print("Strafe right")
rm.right(0.6); rm.stop(); time.sleep(0.25)

print("Diagonal left")
rm.diagonal_left(0.7); rm.stop(); time.sleep(0.25)

print("Diagonal right")
rm.diagonal_right(0.7); rm.stop(); time.sleep(0.25)

print("Movement demo complete.")


In [4]:
import eyes_lib as elib
importlib.reload(elib)

eyes = elib.get_eyes() if hasattr(elib, "get_eyes") else getattr(elib, "eyes", None) or elib.Eyes()
print("Eyes demo")

eyes.set_both(255, 0, 0); time.sleep(0.25)
eyes.set_both(0, 255, 0); time.sleep(0.25)
eyes.set_both(0, 0, 255); time.sleep(0.25)
eyes.blink(color=(253, 208, 0), period_s=0.18, duration_s=1.0)
eyes.wink("left",  color=(255, 255, 255), on_s=0.12, off_s=0.10, repeats=2)
eyes.wink("right", color=(255, 255, 255), on_s=0.12, off_s=0.10, repeats=2)
eyes.off()

print("Eyes demo complete.")


In [5]:
import camera_lib as camlib
importlib.reload(camlib)

cam = camlib.get_cam() if hasattr(camlib, "get_cam") else getattr(camlib, "cam", None) or camlib.Camera()
print("Camera demo")

cam.nod(); time.sleep(0.2)
cam.shake(); time.sleep(0.2)
cam.wiggle(cycles=1, amplitude=150); time.sleep(0.2)
cam.center_all()

print("Camera demo complete.")


In [6]:
import tts_lib as tts
importlib.reload(tts)

print("TTS demo")
tts.warm_piper("ryan")

path = tts.pre_synth(
    "Kia ora! Quick audio test from Turbo.",
    voice="ryan",
    length_scale="0.98",
    sentence_silence="0.08",
)
print("Playing:", path)

play = getattr(tts, "play_path_async", None) or tts.play_wav_async
p = play(path, device=None)
time.sleep(0.2)
print("TTS queued.")


In [7]:
# Always finish safe
import robot_moves as rm
rm.stop()
print("Stopped.")
